In [396]:
import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import datawig
from sklearn.model_selection import train_test_split
from datawig import Imputer
from datawig.column_encoders import *
from datawig.mxnet_input_symbols import *
from datawig.utils import random_split
import pandas as pd


In [397]:
df = pd.read_csv("./natsurvey/national_survey.csv")

/Users/nathan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (73,119) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [398]:
ind_df = pd.read_csv("./Ind2018/Ind2018.csv")

In [399]:
ind_df = ind_df.replace(r'\s+|^$', np.nan, regex=True)

### Reading in Data

In [400]:
nat_df = pd.read_csv('./Data/nat_df_uncleaned.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

/Users/nathan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (106) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Removing Uniform Null Values 
In this survey there are about 10 questions that are uniform across the four versions of the survery. I need to remoeve the ones that are null to make a cleaner dataset. 

In [401]:
df.head(2)

,ID,HOU,SEN,GOV,ABORTION18,AGE,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BOYSGIRLS,BREAK18,BREAK18A,BREAK18B,BRNAGAIN,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3
0,25480,The Republican candidate,The Democratic candidate,The Republican candidate,,30-44,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,,,,Complete,,,,,,,23,,,,,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White women college graduates,,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,
1,25481,The Democratic candidate,The Democratic candidate,The Democratic candidate,,30-44,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,,,,Complete,,,,,,,40,,,,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,


In [402]:
df['EDUC18'].value_counts()

Bachelor's degree (BA or BS)                                                   5135
Attended college but received no degree                                        4539
An advanced degree after a bachelor's degree (such as JD, MA, MBA, MD, PhD)    3829
Never attended college                                                         2934
Associate's degree (AA or AS)                                                  2485
                                                                                299
Name: EDUC18, dtype: int64

In [403]:
df = df.replace(' ', np.nan)

In [404]:
df.head(2)

,ID,HOU,SEN,GOV,ABORTION18,AGE,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BOYSGIRLS,BREAK18,BREAK18A,BREAK18B,BRNAGAIN,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3
0,25480,The Republican candidate,The Democratic candidate,The Republican candidate,NaN,30-44,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,NaN,NaN,NaN,Complete,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White women college graduates,NaN,NaN,NaN,Favorable,NaN,NaN,NaN,NaN,NaN,NaN,Minor changes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Gun policy,NaN,No,NaN,NaN,Approve,NaN,NaN,18-44,18-64,NaN,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,NaN,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,NaN,NaN,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,NaN,NaN,NaN,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,NaN,Telephone survey,NaN,NaN,NaN,Before October,NaN,NaN,NaN,NaN,NaN,Too tough,NaN,Version 2,NaN,NaN,NaN,1.15386927127838,White Catholics,NaN,NaN,NaN,NaN,NaN,NaN
1,25481,The Democratic candidate,The Democratic candidate,The Democratic candidate,NaN,30-44,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,NaN,NaN,NaN,Complete,NaN,NaN,NaN,NaN,NaN,NaN,40,NaN,NaN,NaN,NaN,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,NaN,NaN,NaN,Favorable,NaN,NaN,NaN,NaN,NaN,NaN,Major changes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Health care,NaN,Yes,NaN,NaN,Approve,NaN,NaN,18-44,18-64,NaN,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,NaN,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,NaN,NaN,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,NaN,Justified,Male,Latino men,NaN,NaN,NaN,Suburbs,Suburbs,English,California,Other,NaN,Telephone survey,NaN,NaN,NaN,Before October,NaN,NaN,NaN,NaN,NaN,Too tough,NaN,Version 2,NaN,NaN,NaN,2.15388107299805,Non-white,NaN,NaN,NaN,NaN,NaN,NaN


In [405]:
df.isnull().sum()

ID                       0
HOU                    319
SEN                   5045
GOV                   5157
ABORTION18           14485
AGE                    123
AGE3                   123
AGE8                   123
AGE45                  123
AGE49                  123
AGE60                  123
AGE65                  123
AGEBLACK               437
AGEBYRACE              437
AGEBYRACE08            524
ATTEND16             14676
ATTEND16B            14676
ATTREL               15087
BACKSIDE                 0
BOYSGIRLS            14650
BREAK18              15173
BREAK18A             15173
BREAK18B             15173
BRNAGAIN             14598
CALL                  3400
CDNUM                    0
CHILD18              14650
CHILDBOY             14650
CHILDGIRL            14650
CONTROLHOUSE         14485
EDUC18                 299
EDUCCOLL               299
EDUCHS                 299
EDUCWHITE              621
EDUCWHITEBYSEX         637
ELECTSECFOREIGN18    14485
FAVDEM2              14635
F

### We need to base our predicitions based on columns wuth no missing values. 
##### The list of columns with uniform values:
1. CDNUM
2. EDUC18
3. AGE8
4. HOU
5. SIZEPLAC
6. SEX
7. REGION
8. PHIL3
9. PARTYID
10. SEN 
12. GOV
13. PRECINCT


In [406]:
df.shape

(19221, 127)

In [407]:
df.isnull().sum()

ID                       0
HOU                    319
SEN                   5045
GOV                   5157
ABORTION18           14485
AGE                    123
AGE3                   123
AGE8                   123
AGE45                  123
AGE49                  123
AGE60                  123
AGE65                  123
AGEBLACK               437
AGEBYRACE              437
AGEBYRACE08            524
ATTEND16             14676
ATTEND16B            14676
ATTREL               15087
BACKSIDE                 0
BOYSGIRLS            14650
BREAK18              15173
BREAK18A             15173
BREAK18B             15173
BRNAGAIN             14598
CALL                  3400
CDNUM                    0
CHILD18              14650
CHILDBOY             14650
CHILDGIRL            14650
CONTROLHOUSE         14485
EDUC18                 299
EDUCCOLL               299
EDUCHS                 299
EDUCWHITE              621
EDUCWHITEBYSEX         637
ELECTSECFOREIGN18    14485
FAVDEM2              14635
F

In [408]:
df_train, df_test = random_split(df, split_ratios=[0.8, 0.2])

In [409]:
data_cols = [CategoricalEncoder('AGE8'),
                     CategoricalEncoder('AGEBYRACE'),
                     CategoricalEncoder('EDUC18'),
                     CategoricalEncoder('PARTY'),
                     CategoricalEncoder('PARTYID'),
                     CategoricalEncoder('PHIL3')]
label_cols = [CategoricalEncoder('SEN')]
data_f_cols =[NumericalFeaturizer('AGE8'),
                        NumericalFeaturizer('AGEBYRACE'),
                        NumericalFeaturizer('EDUC18'),
                        NumericalFeaturizer('PARTY'),
                        NumericalFeaturizer('PARTYID'),
                        NumericalFeaturizer('PHIL3'),]

In [410]:
imputer_1 = Imputer(
    data_featurizers= data_f_cols,
    label_encoders= label_cols,
    data_encoders= data_cols,
    output_path='imputer_model'
)

In [411]:
imputer_1.fit(train_df=df_train )
predictions = imputer.predict(df_test)

2019-11-20 12:01:16,471 [INFO]  CategoricalEncoder for column SEN                                found only 90 occurrences of value Did not vote
2019-11-20 12:01:17,222 [INFO]  
========== start: fit model
2019-11-20 12:01:17,224 [WARNING]  Already bound, ignoring bind()
2019-11-20 12:01:17,721 [INFO]  Epoch[0] Batch [0-320]	Speed: 10446.36 samples/sec	cross-entropy=0.710125	SEN-accuracy=0.735981
2019-11-20 12:01:18,174 [INFO]  Epoch[0] Train-cross-entropy=0.655080
2019-11-20 12:01:18,175 [INFO]  Epoch[0] Train-SEN-accuracy=0.778997
2019-11-20 12:01:18,176 [INFO]  Epoch[0] Time cost=0.949
2019-11-20 12:01:18,182 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-11-20 12:01:18,242 [INFO]  Epoch[0] Validation-cross-entropy=0.680514
2019-11-20 12:01:18,243 [INFO]  Epoch[0] Validation-SEN-accuracy=0.750000
2019-11-20 12:01:18,725 [INFO]  Epoch[1] Batch [0-320]	Speed: 10717.91 samples/sec	cross-entropy=0.615071	SEN-accuracy=0.787773
2019-11-20 12:01:19,189 [INFO]  Epoch[1] 

2019-11-20 12:01:33,406 [INFO]  Epoch[14] Train-cross-entropy=0.552552
2019-11-20 12:01:33,407 [INFO]  Epoch[14] Train-SEN-accuracy=0.800940
2019-11-20 12:01:33,409 [INFO]  Epoch[14] Time cost=1.102
2019-11-20 12:01:33,415 [INFO]  Saved checkpoint to "imputer_model/model-0014.params"
2019-11-20 12:01:33,471 [INFO]  Epoch[14] Validation-cross-entropy=0.582323
2019-11-20 12:01:33,473 [INFO]  Epoch[14] Validation-SEN-accuracy=0.789613
2019-11-20 12:01:34,010 [INFO]  Epoch[15] Batch [0-320]	Speed: 9604.38 samples/sec	cross-entropy=0.556361	SEN-accuracy=0.802960
2019-11-20 12:01:34,500 [INFO]  Epoch[15] Train-cross-entropy=0.551507
2019-11-20 12:01:34,501 [INFO]  Epoch[15] Train-SEN-accuracy=0.800157
2019-11-20 12:01:34,502 [INFO]  Epoch[15] Time cost=1.028
2019-11-20 12:01:34,508 [INFO]  Saved checkpoint to "imputer_model/model-0015.params"
2019-11-20 12:01:34,557 [INFO]  Epoch[15] Validation-cross-entropy=0.580911
2019-11-20 12:01:34,559 [INFO]  Epoch[15] Validation-SEN-accuracy=0.790493


2019-11-20 12:01:48,433 [INFO]  Epoch[28] Validation-SEN-accuracy=0.794014
2019-11-20 12:01:48,882 [INFO]  Epoch[29] Batch [0-320]	Speed: 11468.56 samples/sec	cross-entropy=0.546476	SEN-accuracy=0.805685
2019-11-20 12:01:49,336 [INFO]  Epoch[29] Train-cross-entropy=0.540117
2019-11-20 12:01:49,337 [INFO]  Epoch[29] Train-SEN-accuracy=0.803096
2019-11-20 12:01:49,339 [INFO]  Epoch[29] Time cost=0.905
2019-11-20 12:01:49,345 [INFO]  Saved checkpoint to "imputer_model/model-0029.params"
2019-11-20 12:01:49,396 [INFO]  Epoch[29] Validation-cross-entropy=0.568928
2019-11-20 12:01:49,397 [INFO]  Epoch[29] Validation-SEN-accuracy=0.793134
2019-11-20 12:01:49,864 [INFO]  Epoch[30] Batch [0-320]	Speed: 11050.87 samples/sec	cross-entropy=0.545705	SEN-accuracy=0.806464
2019-11-20 12:01:50,416 [INFO]  Epoch[30] Train-cross-entropy=0.539419
2019-11-20 12:01:50,418 [INFO]  Epoch[30] Train-SEN-accuracy=0.803096
2019-11-20 12:01:50,419 [INFO]  Epoch[30] Time cost=1.020
2019-11-20 12:01:50,425 [INFO]  

2019-11-20 12:02:03,704 [INFO]  Epoch[43] Time cost=0.936
2019-11-20 12:02:03,710 [INFO]  Saved checkpoint to "imputer_model/model-0043.params"
2019-11-20 12:02:03,760 [INFO]  Epoch[43] Validation-cross-entropy=0.565957
2019-11-20 12:02:03,761 [INFO]  Epoch[43] Validation-SEN-accuracy=0.794014
2019-11-20 12:02:04,247 [INFO]  Epoch[44] Batch [0-320]	Speed: 10594.40 samples/sec	cross-entropy=0.536905	SEN-accuracy=0.809190
2019-11-20 12:02:04,695 [INFO]  Epoch[44] Train-cross-entropy=0.530885
2019-11-20 12:02:04,697 [INFO]  Epoch[44] Train-SEN-accuracy=0.807602
2019-11-20 12:02:04,698 [INFO]  Epoch[44] Time cost=0.936
2019-11-20 12:02:04,705 [INFO]  Saved checkpoint to "imputer_model/model-0044.params"
2019-11-20 12:02:04,753 [INFO]  Epoch[44] Validation-cross-entropy=0.565864
2019-11-20 12:02:04,754 [INFO]  Epoch[44] Validation-SEN-accuracy=0.794014
2019-11-20 12:02:05,222 [INFO]  Epoch[45] Batch [0-320]	Speed: 11013.08 samples/sec	cross-entropy=0.536481	SEN-accuracy=0.809385
2019-11-20 

2019-11-20 12:02:18,699 [INFO]  Epoch[58] Batch [0-320]	Speed: 10642.59 samples/sec	cross-entropy=0.531633	SEN-accuracy=0.809774
2019-11-20 12:02:19,250 [INFO]  Epoch[58] Train-cross-entropy=0.526029
2019-11-20 12:02:19,252 [INFO]  Epoch[58] Train-SEN-accuracy=0.809169
2019-11-20 12:02:19,253 [INFO]  Epoch[58] Time cost=1.038
2019-11-20 12:02:19,260 [INFO]  Saved checkpoint to "imputer_model/model-0058.params"
2019-11-20 12:02:19,314 [INFO]  Epoch[58] Validation-cross-entropy=0.564649
2019-11-20 12:02:19,315 [INFO]  Epoch[58] Validation-SEN-accuracy=0.796655
2019-11-20 12:02:19,857 [INFO]  Epoch[59] Batch [0-320]	Speed: 9502.79 samples/sec	cross-entropy=0.531349	SEN-accuracy=0.809969
2019-11-20 12:02:20,451 [INFO]  Epoch[59] Train-cross-entropy=0.525815
2019-11-20 12:02:20,454 [INFO]  Epoch[59] Train-SEN-accuracy=0.808777
2019-11-20 12:02:20,456 [INFO]  Epoch[59] Time cost=1.139
2019-11-20 12:02:20,468 [INFO]  Saved checkpoint to "imputer_model/model-0059.params"
2019-11-20 12:02:20,55

2019-11-20 12:02:34,538 [INFO]  Saved checkpoint to "imputer_model/model-0072.params"
2019-11-20 12:02:34,588 [INFO]  Epoch[72] Validation-cross-entropy=0.562127
2019-11-20 12:02:34,589 [INFO]  Epoch[72] Validation-SEN-accuracy=0.798415
2019-11-20 12:02:35,057 [INFO]  Epoch[73] Batch [0-320]	Speed: 11003.30 samples/sec	cross-entropy=0.528271	SEN-accuracy=0.809385
2019-11-20 12:02:35,528 [INFO]  Epoch[73] Train-cross-entropy=0.523756
2019-11-20 12:02:35,529 [INFO]  Epoch[73] Train-SEN-accuracy=0.810149
2019-11-20 12:02:35,530 [INFO]  Epoch[73] Time cost=0.940
2019-11-20 12:02:35,536 [INFO]  Saved checkpoint to "imputer_model/model-0073.params"
2019-11-20 12:02:35,584 [INFO]  Epoch[73] Validation-cross-entropy=0.562036
2019-11-20 12:02:35,585 [INFO]  Epoch[73] Validation-SEN-accuracy=0.800176
2019-11-20 12:02:36,062 [INFO]  Epoch[74] Batch [0-320]	Speed: 10803.38 samples/sec	cross-entropy=0.528209	SEN-accuracy=0.808995
2019-11-20 12:02:36,503 [INFO]  Epoch[74] Train-cross-entropy=0.52365

2019-11-20 12:02:50,302 [INFO]  Epoch[87] Train-cross-entropy=0.522771
2019-11-20 12:02:50,303 [INFO]  Epoch[87] Train-SEN-accuracy=0.811324
2019-11-20 12:02:50,304 [INFO]  Epoch[87] Time cost=0.958
2019-11-20 12:02:50,310 [INFO]  Saved checkpoint to "imputer_model/model-0087.params"
2019-11-20 12:02:50,360 [INFO]  No improvement detected for 3 epochs compared to 0.5609937950339116 last error obtained: 0.5609998142635318, stopping here
2019-11-20 12:02:50,361 [INFO]  
========== done (93.13907384872437 s) fit model
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average,

IndexError: index 10243 is out of bounds for axis 0 with size 10237

In [ ]:
data_encoder_cols = [CategoricalEncoder('AGE8'),
                     CategoricalEncoder('AGEBYRACE'),
                     CategoricalEncoder('EDUC18'),
                     CategoricalEncoder('PARTY'),
                     CategoricalEncoder('PARTYID'),
                     CategoricalEncoder('PHIL3'),
                     CategoricalEncoder('PTYIDEO7'), 
                     CategoricalEncoder('SEN'),
                     CategoricalEncoder('SEX'), 
                     CategoricalEncoder('SIZEPLAC'),
                     CategoricalEncoder('STANUM'), 
                     CategoricalEncoder('PRECINCT'),
                     CategoricalEncoder('CDNUM')
                    ]

label_encoder_cols = [CategoricalEncoder('TRUMP2')]
data_featurizer_cols = [NumericalFeaturizer('AGE8'),
                        NumericalFeaturizer('AGEBYRACE'),
                        NumericalFeaturizer('EDUC18'),
                        NumericalFeaturizer('PARTY'),
                        NumericalFeaturizer('PARTYID'),
                        NumericalFeaturizer('PHIL3'), 
                        NumericalFeaturizer('PTYIDEO7'),
                        NumericalFeaturizer('SEN'),
                        NumericalFeaturizer('SEX'),
                        NumericalFeaturizer('SIZEPLAC'),
                        NumericalFeaturizer('STANUM'),
                        NumericalFeaturizer('PRECINCT'),
                        NumericalFeaturizer('CDNUM')
                       ]

In [ ]:
imputer = Imputer(
    data_featurizers=data_featurizer_cols,
    label_encoders=label_encoder_cols,
    data_encoders=data_encoder_cols,
    output_path='imputer_model'
)



In [ ]:
imputer.fit(train_df=df_train )
predictions = imputer.predict(df_test)

In [ ]:
predictions

In [ ]:
df.EDUC18.value_counts()

In [ ]:
df.BACKSIDE.value_counts()

In [ ]:
df.head(2)

### Labeling Unlabeled Features 

In [ ]:
def label_converter(nat_df, string): 
    labelencoder = LabelEncoder()
    labelencoder.fit(nat_df[string])
    nat_df[string] = labelencoder.transform(nat_df[string])
    return nat_df

In [ ]:
nat_df['AGEBYRACE'].value_counts()

In [ ]:
nat_df['AGE8'].value_counts()

In [ ]:
label_converter

### Dropping unnecessary columns 

In [ ]:
#df.drop('column_name', axis=1, inplace=True)

In [ ]:
nat_df.shape

In [ ]:
nat_df.tail(3)

In [ ]:
nat_df.drop(['AGE3', 'AGE45', 'AGE65', 'AGEBLACK', 'AGEBYRACE08', 'AGE49', 'AGEBYRACE', 'AGE60',
                            'ATTEND16B', 'ATTREL', 'BREAK18', 'CALL',
                            'CHILDBOY', 'CHILDGIRL', 'EDUCCOLL', 'EDUCHS', 'EDUCWHITE',
                            'BREAK18', 'BREAK18A', 'BREAK18B', 'EDUCWHITEBYSEX', 'FAVDEM2',
                            'FAVREP2', 'FTVOTER18', 'GUNOWNER12','INC30K', 'INC50K',
                            'INCOME3', "INCWHITE", "LGBT", "MARRIED", 'NEC2', 
                            'PARTYBLACK', 'PARTYWHITE', 'OVER45', 'OVER65', 'PARTYGENDER', 
                            'QRACEAK', 'QTYPE', 'RACE', 'RACE2B', 'RACEAI', 'RELIGN18', 'RELWHITE',
                            'REL5', 'SEXBYRACE', 'SIZEPLC3', 'SPANISHQ', 'TELEPOLL', 'TIME18HOU_1', 
                            'TIME18HOUMONTH_1', 'TIME18HOUWEEK_1', 'TIME18HOU_2', 'VETVOTER', 'WHITEREL',
                            'WHNCLINC', 'WHTEVANG', 'WOMEN2', 'WPROTBRN', "WPROTBRN3"], axis = 1, 
                            inplace = True)

In [ ]:
nat_df.head(2)

In [ ]:
# Omit VOTEISS18, VETVOTER, WHITEREL, WHTEVANG, WOMEN, WOMEN2, WPROTBRN, WPROTBRN3, ID,
# AGE3, AGE45, AGE60, AGE65, AGEBLACK, AGEBYRACE08, ATTEND16B, ATTREL, BACKSIDE Not complete
# BREAK18, BRNAGAIN, CALL, CHILDBOY, CHILDGIRL, EDUCCOLL, EDUCHS, EDUCWHITE, EDUCWHITEBYSEX,
# FAVDEM2, FAVREP2, FTVOTER18, GUNOWNER12, INC100k, INC30k, INC50k, INCOME3, 
# INCWHITE, LGBT, MARRIED, NEC2, QRACE3, PARTYBLACK, PARTYBLACK, PARTYWHITE, QTYPE
# QRACEAK, QRACEHI, RACE, RACE2B, RELWHITE, SEXUALHARASSB, SIZEPLC3, SPANISHQ, TELEPOLL, 
# TIME18HOU_1, TIME18HOUMONTH_1, TIME18HOUWEEK_1, TRUMP2,WHNCLINC, WHTEVANG, WOMEN2,WPROTBRN, 
# WPROTBRN3
# Later #VERSION, WEIGHTS

In [ ]:
ind_df.head(2)

In [ ]:
 CategoricalEncoder('CDNUM'),
                     CategoricalEncoder('PRECINCT'),
                     CategoricalEncoder('GOV'),
                     CategoricalEncoder('HOU'),
                 NumericalFeaturizer('PRECINCT'),
                        NumericalFeaturizer('CDNUM'),
                        NumericalFeaturizer('GOV'),
                        NumericalFeaturizer('HOU'),

In [ ]:
train, test = random_split(ind_df, split_ratios=[0.8, 0.2] )

In [ ]:
data_encoder_cols = [CategoricalEncoder('AGE8'),
                     CategoricalEncoder('AGEBYRACE'),
                     CategoricalEncoder('EDUC18'),
                     CategoricalEncoder('INCOME18GEN'),
                     CategoricalEncoder('ISSUE18'),
                     CategoricalEncoder('PARTY'),
                     CategoricalEncoder('PARTYID'),
                     CategoricalEncoder('PHIL3'),
                     CategoricalEncoder('PTYIDEO7'), 
                     CategoricalEncoder('SEN'),
                     CategoricalEncoder('SEX'), 
                     CategoricalEncoder('SIZEPLAC'),
                      
                    ]
label_encoder_cols = [CategoricalEncoder('HEALTHCARE18')]
data_featurizer_cols = [NumericalFeaturizer('AGE8'),
                        NumericalFeaturizer('AGEBYRACE'),
                        NumericalFeaturizer('EDUC18'),
                        NumericalFeaturizer('INCOME18GEN'),
                        NumericalFeaturizer('ISSUE18'),
                        NumericalFeaturizer('PARTY'), 
                        NumericalFeaturizer('PARTYID'),
                        NumericalFeaturizer('PHIL3'), 
                        NumericalFeaturizer('PTYIDEO7'),
                        NumericalFeaturizer('SEN'),
                        NumericalFeaturizer('SEX'),
                        NumericalFeaturizer('SIZEPLAC'),
                       
                        ]

In [ ]:
imputer = Imputer(
    data_featurizers= data_featurizer_cols,
    label_encoders= label_encoder_cols,
    data_encoders = data_encoder_cols,
    output_path= 'imputer_model'
)

In [ ]:
imputer.fit(train_df = train)
predictions = imputer.predict(test)

In [ ]:
predictions, metrics = imputer.transform_and_compute_metrics(test)

In [ ]:
metrics

In [ ]:
ind_df.head()

In [ ]:
ind_df['VERSION'].value_counts()

In [ ]:
predictions, metrics = imputer.transform_and_compute_metrics(test)

In [412]:
metrics

{'TRUMP2': {'class_counts': [('Strongly or somewhat disapprove', 690),
   ('Strongly or somewhat approve', 418),
   ('Omit', 6)],
  'avg_precision': 0.5762223940878233,
  'avg_recall': 0.5163627117860528,
  'avg_f1': 0.5271939272708622,
  'avg_accuracy': 0.8204667863554758,
  'weighted_precision': 0.8401180133509024,
  'weighted_recall': 0.8204667863554758,
  'weighted_f1': 0.8061486879765788,
  'weighted_accuracy': 0.8204667863554758,
  'class_precision': [0.7842227378190255, 0.9444444444444444, 0.0],
  'class_recall': [0.9797101449275363, 0.569377990430622, 0.0],
  'class_f1': [0.8711340206185568, 0.7104477611940299, 0.0],
  'class_accuracy': 0.8204667863554758,
  'num_classes': 3,
  'num_applicable_rows': 1114,
  'confusion_matrix': [('Strongly or somewhat disapprove',
    [('Strongly or somewhat disapprove', 676),
     ('Strongly or somewhat approve', 14)]),
   ('Strongly or somewhat approve',
    [('Strongly or somewhat approve', 238),
     ('Strongly or somewhat disapprove', 180)

In [133]:
prob_dict = imputer.predict_proba(test)
prob_dict_topk = imputer.predict_proba_top_k(test, top_k=5)

In [134]:
prob_dict_topk

{'HEALTHCARE18': [[(' ', 0.7625036),
   ('Major changes', 0.16716667),
   ('Minor changes', 0.06939898),
   ('No changes at all', 0.00079899206),
   ('Omit', 0.0001317096)],
  [('Major changes', 0.6484912),
   ('Minor changes', 0.24287705),
   ('No changes at all', 0.063981175),
   (' ', 0.041147366),
   ('Omit', 0.0035032318)],
  [(' ', 0.98244053),
   ('Major changes', 0.009138335),
   ('Minor changes', 0.008226803),
   ('Omit', 0.00011202672),
   ('No changes at all', 8.225218e-05)],
  [('Major changes', 0.7867498),
   ('Minor changes', 0.1313014),
   (' ', 0.07053144),
   ('No changes at all', 0.011326074),
   ('Omit', 9.116322e-05)],
  [(' ', 0.80096155),
   ('Major changes', 0.12078298),
   ('Minor changes', 0.07680639),
   ('No changes at all', 0.0013408471),
   ('Omit', 0.00010815749)],
  [(' ', 0.9784288),
   ('Major changes', 0.01742324),
   ('Minor changes', 0.0038645384),
   ('No changes at all', 0.00024822247),
   ('Omit', 3.519264e-05)],
  [('Major changes', 0.68822396),


In [136]:
predictions

{'HEALTHCARE18': 0                   
 1      Major changes
 2                   
 3      Major changes
 4                   
            ...      
 468    Major changes
 469                 
 470    Major changes
 471                 
 472    Major changes
 Length: 473, dtype: object}

In [137]:
test.shape

(473, 110)

In [138]:
train.shape

(1892, 110)

In [139]:
train.isnull().sum()

ID            0
SENIN18       0
ABORTION18    0
AGE           0
AGE3          0
             ..
WEIGHT        0
WHNCLINC      0
WHTEVANG      0
WOMEN         0
WOMEN2        0
Length: 110, dtype: int64

In [141]:
ind_df.head(2)

,ID,SENIN18,ABORTION18,AGE,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,BACKSIDE,BRAUNLIB,BRNAGAIN,CALL,CDNUM,CONTROLHOUSE,CONTROLSEN,DONNELLYLIB,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,FAVBRAUN,FAVDEM2,FAVDONNELLY,FAVMCONL,FAVPARTY,FAVPELOSI,FAVPENCE,FAVREP2,FAVSENIN18,FINSIT2,FORTRUMPS,FTVOTER18,GEOCODE,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVIN18,KAVIN18A,KAVIN18B,LATINO,MARRIED,NEC,NEC2,OVER45,OVER65,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QTYPE,QUALSEN18,RACE,RACE2B,RACEAI,REGION,RUSSIA18,SEN,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18SEN_1,TIME18SENMONTH_1,TIME18SENWEEK_1,TIME18SEN_2,TRADE18,TRADEBRAUN,TRADEDONNELLY,TRADEIN18,TRUMP2,TRUMP4,TRUMPIMM,VERSION,VETVOTER,VOTE2016,WEIGHT,WHNCLINC,WHTEVANG,WOMEN,WOMEN2
0,6739,NaN,NaN,45-65,30-59,50-59,45+,50+,45-59,50-64,NaN,NaN,NaN,Complete,NaN,Yes,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-whites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Immigration,NaN,NaN,NaN,NaN,Yes,Good,NaN,45+,18-64,Republican,NaN,NaN,NaN,Republican,NaN,NaN,Conservative,NaN,NaN,NaN,Other,Other,NaN,NaN,Other,Non-white,Other,Midwest,NaN,NaN,Male,NaN,NaN,NaN,NaN,NaN,NaN,Indiana,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.751945,No,NaN,NaN,NaN
1,6740,NaN,NaN,45-65,30-59,50-59,45+,50+,45-59,50-64,NaN,NaN,NaN,Complete,NaN,No,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$100,000",NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,Excellent,NaN,45+,18-64,Republican,NaN,NaN,NaN,Republican,NaN,NaN,Conservative,NaN,NaN,NaN,White,White,NaN,NaN,White,White,White,Midwest,NaN,NaN,Male,NaN,NaN,NaN,NaN,NaN,NaN,Indiana,Other,Helped,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.648171,No,NaN,NaN,NaN


In [ ]:
df.dropna(subset = ['EDUC18'], inplace= True)
df.dropna(subset = ['AGE8'], inplace = True)
df.dropna(subset = ['HOU'], inplace = True)
df.dropna(subset = ['SIZEPLAC'], inplace = True)
df.dropna(subset = ['SEX'], inplace = True)
df.dropna(subset = ['REGION'], inplace = True)
df.dropna(subset = ['PHIL3'], inplace = True)
df.dropna(subset = ['PARTYID'], inplace = True)
df.dropna(subset = ['GOV'], inplace = True)
df.dropna(subset = ['PRECINCT'], inplace = True)
df.dropna(subset = ['AGEBYRACE'], inplace = True)